In [39]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os


In [40]:
#similar images
image1_path = './dataset-dev/plates/GB.292020.A-009.tif'
image2_path = './Users/mac/Desktop/tbi/Semester-Project/data/images/GB.191319946.A-006.tif'
image3_path = './Users/mac/Desktop/tbi/Semester-Project/data/images/GB.425576.A-003.tif'
image4_path = '/Users/mac/Desktop/tbi/Semester-Project/data/images/GB.512507.A-003.tif'

In [41]:


def is_contour_inside(contour1, contour2):
    x1, y1, w1, h1 = cv2.boundingRect(contour1)  # inside
    x2, y2, w2, h2 = cv2.boundingRect(contour2)  # outside
    if x1 >= x2 and y1 >= y2 and x1 + w1 <= x2 + w2 and y1 + h1 <= y2 + h2:
        return True

def bounding_box_to_contour(box):
    x, y, w, h = box
    contour = np.array([
        [x, y],
        [x + w, y],
        [x + w, y + h],
        [x, y + h]
    ])
    return contour

In [42]:
gb = []
for i in os.listdir('./dataset-dev/plates'):
    if i.startswith('GB'):
        gb.append(i)
gb

['GB.112704.A-011.tif',
 'GB.426870.A-007.tif',
 'GB.220420.A-010.tif',
 'GB.544722.A-035.tif',
 'GB.544722.A-021.tif',
 'GB.411884.A-027.tif',
 'GB.411884.A-033.tif',
 'GB.411963.A-008.tif',
 'GB.496119.A-009.tif',
 'GB.272628.A-015.tif',
 'GB.191506686.A-011.tif',
 'GB.545196.A-008.tif',
 'GB.125650.A-013.tif',
 'GB.152368.A-038.tif',
 'GB.191223716.A-012.tif',
 'GB.216640.A-004.tif',
 'GB.171939.A-005.tif',
 'GB.121298.A-007.tif',
 'GB.334584.A-060.tif',
 'GB.359612.A-007.tif',
 'GB.334584.A-048.tif',
 'GB.401900.A-005.tif',
 'GB.401900.A-004.tif',
 'GB.334584.A-049.tif',
 'GB.359612.A-006.tif',
 'GB.334584.A-061.tif',
 'GB.331224.A-006.tif',
 'GB.121298.A-012.tif',
 'GB.171939.A-010.tif',
 'GB.152368.A-011.tif',
 'GB.191417192.A-005.tif',
 'GB.216640.A-005.tif',
 'GB.191223716.A-013.tif',
 'GB.152368.A-039.tif',
 'GB.125650.A-012.tif',
 'GB.191506686.A-010.tif',
 'GB.545196.A-009.tif',
 'GB.272628.A-014.tif',
 'GB.374851.A-004.tif',
 'GB.411963.A-009.tif',
 'GB.411884.A-032.tif',
 

In [43]:
def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_canny = cv2.Canny(img_gray, 0, 250)
    kernel = np.ones((3, 3), np.uint8)
    img_dilate = cv2.dilate(img_canny, kernel, iterations=4)
    return img_dilate

#first phase of segmentation to remove outer border present in images
def first_phase(img_path):
    img = cv2.imread(img_path)
    img_copy = np.copy(img)
    
    processed_img = process(img_copy)
    contours, hierarchy = cv2.findContours(processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    filter_contours = [ct for ct in contours if cv2.contourArea(ct) > 3000] #filter out small contours
    areas = []
    original_size = img_copy.shape
    #print(len(filter_contours))
    for ct in filter_contours:
        x, y, w, h = cv2.boundingRect(ct)
        areas.append(w*h)
        
    #print(len(areas))
    biggest = filter_contours[areas.index(max(areas))]
    #print(cv2.arcLength(biggest, True))
    #print(cv2.arcLength(biggest, False))
    
    epsilon = 0.02 * cv2.arcLength(biggest, False)
    approx = cv2.approxPolyDP(biggest, epsilon, False)
    #cv2.drawContours(img_copy, [approx], -1, (0, 0, 255), 2)
    #cv2.polylines(img_copy, [approx], isClosed=False, color=(0, 255, 0), thickness=2)
    hull =cv2.convexHull(biggest)
    #print(hull)
    cv2.drawContours(img_copy, [hull], -1, (0, 0, 255), 2)
    #k = cv2.isContourConvex(biggest)
    #print(k)
    #cv2.drawContours(img_copy,[biggest], -1, (0, 0, 255), 2)
    oldx, oldy, oldw, oldh = cv2.boundingRect(biggest)
    #paddingx= oldw* 1//30
    #paddingy= oldh* 1//20
    paddingx= 100
    paddingy= 75
    new_w = oldw - paddingx
    new_h = oldh - paddingy
    new_x = oldx + paddingx
    new_y = oldy + paddingy
    biggest = img_copy[new_y:oldy+new_h, new_x:oldx+new_w]
    # Create a white background image with the same size as the original
    white_background = np.ones((original_size[0], original_size[1], 3), dtype=np.uint8) * 255
    
    # Calculate the center position
    #center_x = (original_size[1] - biggest.shape[1]) // 2
    #center_y = (original_size[0] - biggest.shape[0]) // 2
    
    # Place the ROI in the center of the white background
    white_background[new_y:new_y+biggest.shape[0], new_x:new_x+biggest.shape[1]] = biggest
    #for ct in filter_contours:
        #x, y, w, h = cv2.boundingRect(ct)
        #cv2.rectangle(img_copy, (x+70, y+70), (x + w-50, y + h-50), (0, 0, 255), 2)
    return img_copy, filter_contours,white_background
        
    
def second_phase(img_first_phase,img_original):
    img_copy = np.copy(img_first_phase)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(process(img_copy), connectivity=8, ltype=cv2.CV_32S)
    contours = []
    #area = img_copy.shape[0] * img_copy.shape[1]
    for i in range(1, num_labels):
        x, y, w, h, _ = stats[i]
        box = (x, y, w, h)
        contour = bounding_box_to_contour(box)
        contours.append(contour)
        
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > 3000]
    # get rid of small contours
    sorted_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)
    
        
    for idx,contour in enumerate(sorted_contours):
        if idx == 0:
            #x, y, w, h = cv2.boundingRect(contour)
            #cv2.rectangle(img_original, (x, y), (x + w, y + h), (0, 0, 255), 2)
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img_original,[box],0,(0,0,255),2)   
        else:
            #x, y, w, h = cv2.boundingRect(contour)
            res = False
            for i in range(idx):
                if is_contour_inside(contour, sorted_contours[i]):
                    res = True
                    break
            if not res: #the contour is not inside any of the previous contours
                #x, y, w, h = cv2.boundingRect(contour)
                #cv2.rectangle(img_original, (x, y), (x + w, y + h), (0, 0, 255), 2)
                rect = cv2.minAreaRect(contour)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(img_original,[box],0,(0,0,255),2)
                


    return img_copy, sorted_contours,img_original
    
    




In [44]:
for i in gb[10:20]:
    img_copy, _,figure = first_phase("./dataset-dev/plates/"+i)
    #print(k)
    cv2.imshow("phase 1", img_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    _, _,segmented = second_phase(figure,img_copy)
    cv2.imshow("phase 2", segmented)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [45]:
# Function to add padding to each corner of the box
def add_padding_to_box(box, padding_values):
    # Get the center of the original box
    center = np.mean(box, axis=0)
    
    # Initialize the new box with the same shape
    padded_box = np.copy(box).astype(np.float64)
    
    # Add padding to each corner
    for i in range(4):
        # Vector from center to the corner
        vector = padded_box[i] - center
        # Normalize the vector
        unit_vector = vector / np.linalg.norm(vector)
        # Apply padding along the direction of the vector
        padded_box[i] += padding_values[i] * unit_vector
        
    return np.int0(padded_box)

In [59]:
def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_canny = cv2.Canny(img_gray, 0, 150)
    kernel = np.ones((3, 3), np.uint8)
    img_dilate = cv2.dilate(img_canny, kernel, iterations=4)
    #erode

    return img_dilate

# First phase of segmentation to remove outer border present in images
def polygone_approx(img_path):
    img = cv2.imread(img_path)
    img_copy = np.copy(img)
    
    processed_img = process(img_copy)
    contours, _ = cv2.findContours(processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filter_contours = [ct for ct in contours if cv2.contourArea(ct) > 3000]  # filter out small contours
    areas = []
    
    
    for ct in filter_contours:
        _, _, w, h = cv2.boundingRect(ct)
        areas.append(w * h)
    
    biggest = filter_contours[areas.index(max(areas))]
    
    
    # Approximate the contour to a polygon
    #epsilon = 0.05 * cv2.arcLength(biggest, False)
    #approx = cv2.approxPolyDP(biggest, epsilon, False)
    #print(approx)
    
    #make the y value of the top left point in the contour the same as the y value of the top right point of the contour
    #approx[0][0][1] = approx[1][0][1]
    #approx[0][0][0] = approx[1][0][0]
    
    hull = cv2.convexHull(biggest)
    
    cv2.drawContours(img, [hull], 0, (0, 0, 255), 2)
    #cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    rect = cv2.minAreaRect(hull)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    print(box)
    
    #cv2.drawContours(img,[box],-1,(0,255,0),2) #green min rect
   
    padding_values = [-50.0, -50.0, -50.0, -50.0]

    # Add padding to the box
    padded_box = add_padding_to_box(box, padding_values)
    cv2.drawContours(img,[padded_box],0,(255,0,0),2) #blue box
    # Create a white background image with the same size as the original
    white_background = np.ones((img.shape[0], img.shape[1], 3), dtype=np.uint8) * 255
    
    x, y, w, h = cv2.boundingRect(padded_box)
    roi = img[y:y+h, x:x+w]
    #put roi in the same position as original image in the white background
    white_background[y:y+h, x:x+w] = roi
    
    
    
    
    
    
    #biggest = img[y:y+h, x:x+w]
    #show content inside polygon
    cv2.drawContours(img, [hull], -1, (0, 0, 255), 2) # red hull
    #draw the polygone on image filled with white background the same size as the original image
    #cv2.fillPoly(mask, pts = [approx], color = 255)
    #masked_img = cv2.bitwise_and(img, mask)
    original_size = img.shape
    # Create a white background image with the same size as the original
    #white_background = np.ones((original_size[0], original_size[1], 3), dtype=np.uint8) * 255
    
    #white_background[y:y+h, x:x+w] = biggest

    
    
    # Fill the polygon on the mask
    #cv2.fillPoly(mask, pts = [approx], color = 255)
    #masked_img = cv2.bitwise_and(img, mask)
    
    

    return img, white_background,hull



In [60]:
for i in gb[25:30]:
    img_copy, new_image,figure = polygone_approx("./dataset-dev/plates/"+i)
    cv2.imshow("phase 1", img_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

[[ 644 3188]
 [ 700  401]
 [2374  435]
 [2318 3222]]
[[ 339 2590]
 [ 346   49]
 [2267   54]
 [2260 2595]]
[[  67 2963]
 [ 139  229]
 [1781  272]
 [1709 3006]]
[[ 278  172]
 [2259  148]
 [2291 2736]
 [ 309 2760]]
[[ 970  236]
 [2408  229]
 [2421 2956]
 [ 983 2963]]


In [61]:

import matplotlib.pyplot as plt

# Load the image
image_path = "./dataset-dev/plates/"+ gb[1]

# Function to compute intersection between two lines given in (rho, theta) form
def compute_intersection(line1, line2):
    rho1, theta1 = line1
    rho2, theta2 = line2
    # Coefficients for line 1
    A1 = np.cos(theta1)
    B1 = np.sin(theta1)
    # Coefficients for line 2
    A2 = np.cos(theta2)
    B2 = np.sin(theta2)
    # Calculate determinant
    determinant = A1 * B2 - A2 * B1
    if determinant == 0:
        return None  # Lines are parallel
    # Calculate intersection point
    x = (B2 * rho1 - B1 * rho2) / determinant
    y = (A1 * rho2 - A2 * rho1) / determinant
    return int(np.round(x)), int(np.round(y))


def hough_lines(image_path):
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply edge detection using Canny
    edges = cv2.Canny(gray, 0, 150)

# Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 0.005, np.pi/180, 150)

# Separate lines into vertical and horizontal lines
    vertical_lines = []
    horizontal_lines = []

    if lines is not None:
        for line in lines:
           rho, theta = line[0]
           if abs(theta) < np.pi / 36 or abs(theta - np.pi) < np.pi / 36:  # Vertical lines
              vertical_lines.append((rho, theta))
           elif abs(theta - np.pi/2) < np.pi / 36:  # Horizontal lines
              horizontal_lines.append((rho, theta))

# Sort lines by their rho value (distance from origin)
    vertical_lines.sort(key=lambda x: x[0])
    horizontal_lines.sort(key=lambda x: x[0])

# Keep only the outermost vertical lines (leftmost and rightmost)
    selected_vertical_lines = [vertical_lines[0], vertical_lines[-1]]

    # Keep only the outermost horizontal lines (topmost and bottommost)
    selected_horizontal_lines = [horizontal_lines[0], horizontal_lines[-1]]



# Compute intersections between each pair of vertical and horizontal lines
    intersections = []
    for v_line in selected_vertical_lines:
        for h_line in selected_horizontal_lines:
            point = compute_intersection(v_line, h_line)
            if point is not None:
                intersections.append(point)

    # Draw only the segments between intersections
    clipped_image = np.copy(image)

    # Draw the top horizontal line segment
    cv2.line(clipped_image, intersections[0], intersections[1], (255, 0, 0), 2)

# Draw the bottom horizontal line segment
    cv2.line(clipped_image, intersections[2], intersections[3], (255, 0, 0), 2)

# Draw the left vertical line segment
    cv2.line(clipped_image, intersections[0], intersections[2], (255, 0, 0), 2)

# Draw the right vertical line segment
    cv2.line(clipped_image, intersections[1], intersections[3], (255, 0, 0), 2)
    
    return clipped_image

# Display the result
for i in gb[:10]:
    clipped_image = hough_lines("./dataset-dev/plates/"+i)
    cv2.imshow("Hough Lines", clipped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

